# Minimal example for the package

This notebook was generated from `run_topicmodels.py`. It contains example runs for the `topicmodels` package (SPF, PF, CSPF, TBIP, ...).

Notes: keep the working directory so that relative paths to `data/10k_amazon.csv` work (run from the repository root).

## Prelims

For each of the following models we need to:
- load the data
- create the DTM
- save the vocabulary

In [1]:
pip install tqdm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# --- Imports ---
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as sparse

# Prelims 
 
# ---- Load data ----
df1 = pd.read_csv("data/10k_amazon.csv")

# ---- Define keywords ----
pets = ["dog","cat", "litter", "cats", "dogs", "food", "box", "collar", "water", "pet"]
toys = ["toy", "game", "play", "fun", "old", "son", "year", "loves", "kids", "daughter"]
beauty = ["hair", "skin", "product", "color", "scent", "smell", "used", "dry", "using", "products"]
baby = ["baby", "seat", "diaper", "diapers", "stroller", "bottles", "son", "pump", "gate", "months"]
health = ["product", "like", "razor", "shave", "time", "day", "shaver", "better", "work", "years"]
grocery = ["tea", "taste", "flavor", "coffee", "sauce", "chocolate", "sugar", "eat", "sweet", "delicious"]

keywords = {"pet supplies": pets, "toys games": toys, "beauty": beauty, "baby products": baby,
            "health personal care": health, "grocery gourmet food": grocery}

# --- Create corpus ---
cv = CountVectorizer(stop_words='english', min_df = 2)
cv.fit(df1["Text"])
counts = sparse.csr_matrix(cv.transform(df1["Text"]), dtype = np.float32)
vocab = cv.get_feature_names_out()

## SPF

Minimal example for the Seeded Poisson Factorization model.

In [5]:
from poisson_topicmodels import SPF

# ---- Initialize TM package ----
tm1 = SPF(counts, vocab, keywords, residual_topics = 0, batch_size = 1024)

# ---- Run inference -----
estimated_params = tm1.train_step(num_steps = 1000, lr = 0.1)

# ---- Inspect results ----
estimated_params
topics, e_theta = tm1.return_topics()
beta = tm1.return_beta()
top_words = tm1.return_top_words_per_topic(n = 10)

# --- See loss within inherited metrics object ---
tm1.Metrics.loss

Init loss:   576.3346; Avg loss (last 10 iter):   247.2767:   7%|▋         | 69/1000 [00:59<13:23,  1.16it/s]


KeyboardInterrupt: 

## PF

Minimal example for the Poisson Factorization model.

In [ ]:
from poisson_topicmodels import PF

tm2 = PF(counts, vocab, num_topics = 10, batch_size = 1024)
estimated_params = tm2.train_step(num_steps = 100, lr =0.01)
topics, e_theta = tm2.return_topics()
betas = tm2.return_beta()


## CSPF

Minimal example for the Covariate-informed Seeded Poisson Factorization model.

In [ ]:
from poisson_topicmodels import CSPF

category0 = ["grocery gourmet food", "toys games"]
covariable = df1['Cat1'].apply(lambda x: 0 if x in category0 else 1)
print(covariable[0:10])
print(df1['Cat1'].head(10))

X_design_matrix = pd.DataFrame({'intercept' : np.repeat(1, len(df1)), 'var_infromative' : covariable})

tm3 = CSPf(counts, vocab, keywords, residual_topics = 2, batch_size = 1024, X_design_matrix = X_design_matrix)
estimated_params = tm3.train_step(num_steps = 1000, lr = 0.01)
topics, e_theta = tm3.return_topics()
betas = tm3.return_beta()


## CPF

Minimal example for the Covariate-informed Poisson Factorization model.

In [ ]:
# from poisson_topicmodels import CPF
# tm3 = CPF(counts, vocab, num_topics = 5, batch_size = 1024, X_design_matrix = X_design_matrix)
# svi_batch, svi_state = tm3.train_step(num_steps = 100, lr = 0.01)
# estimated_params = svi_batch.get_params(svi_state)


## TBIP

Minimal example for Text-based ideal point model.

In [ ]:
from poisson_topicmodels import TBIP
df1['speaker'] = np.random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K'], size=len(df1), replace=True)
tm4 = TBIP(counts, vocab, num_topics = 10, authors = df1.speaker, batch_size = 1024)
estimated_params = tm4.train_step(num_steps = 1000, lr = 0.01)

## TVTBIP

Minimal example for the Time-varying Text-based ideal point model.


Already implemented. Add minimal example here.